In [ ]:
from dataset.audio_dataset import LabeledAudioData

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
a = LabeledAudioData('/home/kureta/Music/violin')

In [ ]:
source = pd.DataFrame({
    'x': np.arange(24),
    'y': a.loudness.numpy()[0]
})

In [ ]:
alt.Chart(source).mark_line().encode(
    x='x',
    y='y'
)

In [ ]:
x = a[0:10]

In [ ]:
x['audio'].shape

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
plt.rcParams['figure.figsize'] = [17, 8]

In [2]:
from model.recon import AudioData

/home/kureta/.conda/envs/ddsp-pytorch/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [3]:
dataset = AudioData('/home/kureta/Music/violin/')

Loading presaved dataset...


In [4]:
mag, audio = dataset[0]

In [ ]:
librosa.display.waveplot(audio.numpy())

In [ ]:
librosa.display.specshow(mag.numpy().T)

In [ ]:
len(dataset)

In [8]:
audio.dtype

torch.float32

In [2]:
x = np.random.randn(250)

In [3]:
y = np.fft.fft(x)

In [10]:
x_hat = np.fft.fft(y.conj()) / len(x)

In [13]:
x[10], x_hat[10]

(-0.737085197491792, (-0.737085197491792-1.4578458566500044e-16j))

In [8]:
x.shape

(250,)

In [9]:
y.shape

(250,)